# 📬 Email Digest Generator (Colab + Gemini)
Questo notebook automatizza la lettura di email da una casella Gmail, le invia a Gemini per generare un riassunto e inoltra il digest a un altro indirizzo.


##✨ Funzionalità
- Connessione a Gmail via IMAP
- Filtro per mittente e soggetto
- Riassunto delle email tramite Google Gemini
- Invio del digest tramite SMTP

## ▶️ Esecuzione
- Configura i secret con i tuoi dati (o mettili nel notebook)
- Esegui tutte le celle
- Riceverai un’email riassuntiva con i contenuti selezionati

## 📦 Requisiti
- Account Google con accesso IMAP attivo
- Etichetta o filtro per individuare le newsletter
- Password per app generata da Google
- Accesso a Gemini (Google AI Studio / API key)

In [71]:
!pip install imapclient beautifulsoup4 --quiet

In [72]:
import smtplib                         # Per inviare email via SMTP
import ssl                             # Per creare una connessione sicura SSL
import email                           # Per manipolare messaggi email (MIME, header, payload)
from email.mime.text import MIMEText  # Per costruire messaggi email in formato testo
from email.header import decode_header # Per decodificare oggetti email codificati (es. utf-8=?)
from imapclient import IMAPClient     # Client IMAP semplice per leggere email (es. Gmail)

from google import genai              # Per usare Google Generative AI (se usi PaLM, Gemini ecc.)
from google.colab import userdata     # Per gestire segreti temporanei in Colab (API key, email)

from bs4 import BeautifulSoup         # Per estrarre testo da HTML (es. corpo delle email)

from datetime import datetime         # Per ottenere e formattare la data/ora corrente
import os                             # Per accedere a variabili d’ambiente e percorsi
import subprocess                     # Per eseguire comandi di sistema (es. shell)
import sys                            # Per gestire argomenti di sistema, stderr, path

import re                             # Per lavorare con espressioni regolari (regex)

In [73]:
try:
    import google.generativeai
    print("google-generativeai already available.")
except ImportError:
    print("Installing google-genai...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "google-genai"])

google-generativeai already available.


In [74]:
# --- CONFIGURAZIONE ---
# (Imposta i dati come secret o inseriscili qui dentro)

EMAIL_ACCOUNT = userdata.get('EMAIL_ACCOUNT')
EMAIL_PASSWORD = userdata.get('EMAIL_PASSWORD')
IMAP_SERVER = userdata.get('IMAP_SERVER')
#IMAP_FOLDER = "INBOX"
IMAP_FOLDER = userdata.get('IMAP_FOLDER')
NEWSLETTER_FILTER = [userdata.get('NEWSLETTER_FILTER')]


SYSTEM_PROMPT = """ Quando l’utente incolla il testo di una newsletter:
0. Scarta subito sezioni non informative (saluti iniziali/finali, disclaimer, CTA, link di disiscrizione, promo, footer, social, ecc.).
1. Riassumi il contenuto rilevante in bullet chiari (usa i punti che servono, tipicamente 5-10).
2. Se l’originale è in inglese, traduci solo il tuo riassunto in italiano.
3. NON usare **nessun tipo di formattazione**:
   - niente grassetto
   - niente corsivo
   - niente asterischi, cancelletti, trattini, tabulazioni
   - niente Markdown
   - niente emoji
   - niente simboli speciali
4. Inizia ogni punto elenco solo con un trattino semplice e uno spazio, così: `- `
5. Se noti riferimenti a eventi/prodotti < 30 gg, esegui una ricerca rapida sul web e integra max 2 aggiornamenti, citando la fonte con link breve.
6. Mantieni stile conciso, diretto, zero introduzioni o conclusioni superflue.
7. Evita opinioni non supportate, dati inventati, contenuti confidenziali.

Il risultato sarà inviato via email, quindi deve essere semplice testo leggibile, senza markup.
Ecco il messaggio:

"""

In [75]:
THINK = True
PRO = False

GEMINI_KEY = userdata.get('GEMINI_KEY')

gemini_client = genai.Client(api_key=GEMINI_KEY)
if THINK:
  if PRO :
    gemini_model="gemini-2.5-pro"
  else :
    gemini_model="gemini-2.5-flash"
else:
  gemini_model="gemini-2.0-flash"
print(gemini_model)

gemini-2.5-flash


In [76]:
def decodifica_oggetto(header_raw):
    parti = decode_header(header_raw)
    oggetto = ""
    for testo, encoding in parti:
        if isinstance(testo, bytes):
            oggetto += testo.decode(encoding or "utf-8", errors="ignore")
        else:
            oggetto += testo
    return oggetto.strip()

In [77]:
def estrai_corpo_email(messaggio_email):
    if messaggio_email.is_multipart():
        for parte in messaggio_email.walk():
            content_type = parte.get_content_type()
            if content_type == "text/plain":
                return parte.get_payload(decode=True).decode(errors='ignore')
            elif content_type == "text/html":
                html = parte.get_payload(decode=True).decode(errors='ignore')
                return BeautifulSoup(html, "html.parser").get_text()
    else:
        return messaggio_email.get_payload(decode=True).decode(errors='ignore')
    return ""

In [78]:
def processa_email(prompt):
    response = gemini_client.models.generate_content(
        model=gemini_model,
        contents=SYSTEM_PROMPT + prompt
    )
    return response.text

In [79]:
def invia_riassunto_email(destinatario, contenuto):
    mittente = EMAIL_ACCOUNT
    data = datetime.now().strftime("%d-%m-%Y")
    oggetto = f"Riassunto {data}"

    msg = MIMEText(contenuto, "plain", "utf-8")
    msg["Subject"] = oggetto
    msg["From"] = mittente
    msg["To"] = destinatario

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)
        server.sendmail(mittente, [destinatario], msg.as_string())

In [80]:
print("Ready to go")

Ready to go


In [81]:
# --- CONNESSIONE EMAIL ---
with IMAPClient(IMAP_SERVER, ssl=True) as client:
    client.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)
    client.select_folder(IMAP_FOLDER)

    # Cerca ultime newsletter
    messaggi = client.search([NEWSLETTER_FILTER])
    ultimi_id = messaggi[-10:]

    corpo_email_finale=""

    for uid in ultimi_id:
        raw_message = client.fetch([uid], ["RFC822"])[uid][b"RFC822"]
        messaggio_email = email.message_from_bytes(raw_message)
        corpo = estrai_corpo_email(messaggio_email)

        print("="*80)
        oggetto = decodifica_oggetto(messaggio_email.get("Subject"))
        print(f"📩 Oggetto: {oggetto}")
        print("-" * 80)
        risultato = processa_email(corpo)
        print(risultato)

        corpo_email_finale += "="*80 + "\n"
        corpo_email_finale += f"📩 Oggetto: {oggetto}\n"
        corpo_email_finale += "-" * 80 + "\n"
        corpo_email_finale += risultato + "\n"

📩 Oggetto: Co-op Hackers Stole Passwords 🛒 , Hiding Malware in DNS 🥷 , Go Gets FIPS 140-3🔐
--------------------------------------------------------------------------------
- L'azienda Co-op ha confermato il furto di dati personali di 6.5 milioni di clienti, inclusi nomi, indirizzi e contatti, a seguito di un cyberattacco avvenuto ad aprile.
- Episource, sussidiaria di UnitedHealth, ha subito un attacco ransomware che ha esposto i dati di 5.4 milioni di pazienti, inclusi cartelle cliniche e informazioni assicurative.
- L'app fitness Fitify ha esposto 138.000 foto e dati privati degli utenti tramite un bucket Google Cloud non protetto, molti dei quali molto sensibili.
- Cloudflare ha mitigato un picco record di attacchi DDoS da 7.3 Tb/s nel secondo trimestre 2025, evidenziando un aumento dell'intensità degli attacchi.
- Nuove tattiche di attacco includono l'occultamento di malware nei record DNS TXT per eludere i sistemi di sicurezza e l'evoluzione del loader Matanbuchus 3.0 tramite impe

In [82]:
# Invia risultato via email
if len(ultimi_id) > 0:
  invia_riassunto_email(EMAIL_ACCOUNT, corpo_email_finale)
  print("mail inviata")
else:
  print("Nessuna mail da inviare")

mail inviata


In [83]:
print("Finish!")

Finish!
